# Applying CRISP-DM on the MIMIC-3 Dataset (Part II)
In this notebook, I apply the 3 last phases of the CRISP-DM process:

4. Modeling
5. Evaluation
6. Deployment

<img src="./media/crisp.png" width="30%">


A quick note here is that we prepare the data for training the model (it would be still related to the step 3). However, the main part of this notebook is related to the final 3 steps.

In [70]:
#### Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.model_selection import cross_val_score, train_test_split

%matplotlib inline

#### The small part related to step 3. Data Preparation

In [11]:
database = 'mimiciii'

In [18]:
query = """
with ce as
(
  select
    icustay_id, charttime, itemid, valuenum
  from chartevents
  -- specify what data we want from chartevents
  where itemid in
  (
  211, -- Heart Rate
  618, --	Respiratory Rate
  615 --	Resp Rate (Total)
  )
  -- how did we know heart rates were stored using ITEMID 211? Simple, we looked in D_ITEMS!
  -- Try it for yourself: select * from d_items where lower(label) like '%heart rate%'
)
select
  -- ICUSTAY_ID identifies each unique patient ICU stay
  -- note that if the same person stays in the ICU more than once, each stay would have a *different* ICUSTAY_ID
  -- however, since it's the same person, all those stays would have the same SUBJECT_ID
  ie.icustay_id

  -- this is the outcome of interest: in-hospital mortality
  , max(adm.HOSPITAL_EXPIRE_FLAG) as OUTCOME

  -- this is a case statement - essentially an "if, else" clause
  , min(
      case
        -- if the itemid is 211
        when itemid = 211
          -- then return the actual value stored in VALUENUM
          then valuenum
        -- otherwise, return 'null', which is SQL standard for an empty value
        else null
      -- end the case statement
      end
    ) as HeartRate_Min

    -- note we wrapped the above in "min()"
    -- this takes the minimum of all values inside, and *ignores* nulls
    -- by calling this on our case statement, we are ignoring all values except those with ITEMID = 211
    -- since ITEMID 211 are heart rates, we take the minimum of only heart rates

  , max(case when itemid = 211 then valuenum else null end) as HeartRate_Max
  , min(case when itemid in (615,618) then valuenum else null end) as RespRate_Min
  , max(case when itemid in (615,618) then valuenum else null end) as RespRate_Max
from icustays ie

-- join to the admissions table to get hospital outcome
inner join admissions adm
  on ie.hadm_id = adm.hadm_id

-- join to the chartevents table to get the observations
left join ce
  -- match the tables on the patient identifier
  on ie.icustay_id = ce.icustay_id
  -- and require that the observation be made after the patient is admitted to the ICU
  and ce.charttime >= ie.intime
  -- and *before* their admission time + 1 day, i.e. the observation must be made on their first day in the ICU
  and ce.charttime <= ie.intime + interval '1' day
group by ie.icustay_id
order by ie.icustay_id
"""

data = wr.athena.read_sql_query(query,database)
data.drop('icustay_id', axis=1, inplace=True)

In [19]:
data.head()

,outcome,heartrate_min,heartrate_max,resprate_min,resprate_max
0,0,NaN,NaN,NaN,NaN
1,0,72.0,122.0,14.0,39.0
2,0,62.0,84.0,14.0,27.0
3,0,80.0,104.0,13.0,29.0
4,0,88.0,106.0,13.0,22.0


In [28]:
# We should have +60K rows in our dataset represeting patients from the MIMIC-III dataset
data.shape

(61532, 5)

## 4. Modeling

Here we develop, train and test the model locally.

In [71]:
# move from a data frame into a numpy array
X = data.iloc[:,1:].to_numpy()
y = data['outcome'].to_numpy()

In [72]:
# Cast to numeric types
X = X.astype(float)
y = y.astype(int)

In [73]:
# split data in 80%-20% training/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [74]:
# impute mean for missing values, since we have only numerical values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [75]:
model = LogisticRegression(fit_intercept=True, solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

## 5. Evaluation

We evaluate our model locally, analyzing the test dataset and using AUROC, confusion matrices and accuracy:

In [76]:
# predict class labels for the test set
y_pred = model.predict(X_test)

# generate class probabilities
y_prob = model.predict_proba(X_test)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

print('\nConfusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\nClassification report')
print(classification_report(y_test, y_pred))

Accuracy = 0.8944503128300967
AUROC = 0.6098985793469475

Confusion matrix
[[10956    20]
 [ 1279    52]]

Classification report
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     10976
           1       0.72      0.04      0.07      1331

   micro avg       0.89      0.89      0.89     12307
   macro avg       0.81      0.52      0.51     12307
weighted avg       0.88      0.89      0.85     12307



In [78]:
# evaluate a logistic regression with 5-fold cross-validation
estimator = Pipeline([("imputer", SimpleImputer(missing_values=np.nan,
                                          strategy="mean")),
                      ('scaler', StandardScaler()),
                      ("regression", LogisticRegressionCV(cv=5,
                                                          scoring='roc_auc',
                                                          solver='lbfgs'))])

scores = cross_val_score(estimator
                         , X, y
                         , scoring='roc_auc', cv=5)


print('AUROC for all folds:')
print(scores)
print('Average AUROC across folds:')
print(scores.mean())

AUROC for all folds:
[0.61605425 0.64243601 0.61885965 0.60049902 0.61505218]
Average AUROC across folds:
0.6185802217140426


In [88]:
# train pipeline
estimator.fit(X_train, y_train)

# predict class labels for the test set
y_pred = model.predict(X_test)

# generate class probabilities
y_prob = model.predict_proba(X_test)

# generate evaluation metrics
print('Accuracy = {}'.format(accuracy_score(y_test, y_pred)))
print('AUROC = {}'.format(roc_auc_score(y_test, y_prob[:, 1])))

Accuracy = 0.8944503128300967
AUROC = 0.6098985793469475


In [96]:
# test saving and loading model locally
import joblib

joblib.dump(estimator, os.path.join('./', "model.joblib"))

['./model.joblib']

In [97]:
estimator = joblib.load(os.path.join('./', "model.joblib"))

estimator

Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('regression', LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, i...andom_state=None, refit=True, scoring='roc_auc',
           solver='lbfgs', tol=0.0001, verbose=0))])

## 6. Deployment
We finally deploy in SageMaker, in a managed training and hosting infrastructure:

For more information about SageMaker, get out the [docs](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html).

Remember to add permissions of SageMakerFullAccess to IAM Role of the notebook instance.

In [80]:
# S3 prefix
prefix = 'Scikit-mimic'

import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import numpy as np
import os

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [81]:
# Create directory and write csv
os.makedirs('./data', exist_ok=True)
np.savetxt('./data/mimic.csv', data, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')

In [82]:
# Save data in S3 for training with SageMaker
data_dir = 'data'
train_input = sagemaker_session.upload_data(data_dir, key_prefix="{}/{}".format(prefix, data_dir) )

In [98]:
script_path = 'scikit_learn_mimic.py'

# Simple training without hyperparameters
# The entry script in a python script based on the code developed above
sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={})

In [100]:
sklearn.fit({'train': train_input})

You should see SageMaker spinning up instances for training:

![sm-train](./media/sagemaker-training.png)

In [102]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

You should see SageMaker spinning up instances for hosting the model with an API endpoint:

![sm-train](./media/sagemaker-hosting.png)

![sm-train](./media/sagemaker-hosting-2.png)

In [104]:
# Delete endpoint for avoid costs
sklearn.delete_endpoint()